source ; https://github.com/dglai/WWW20-Hands-on-Tutorial/blob/master/_legacy/advanced_apps/rec/Recommendation.ipynb

# Problem setting

In this tutorial, we demonstarte how graph neural networks can be used for recommendtaion. Here we focus on item-based recommendtaion model. This method in this tutorial recommends items that are similar to the ones purchased by the user. We demonstrate the recommendation model on the MovieLens dataset.

### Get started

DGL can be used with different deep learning frameworks. Currently, DGL can be used with Pytorch and MXNet. Here, we show how DGL works wtih Pytorch.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

When we load DGL. we need to set the DGL backend for one of the deep learning frameworks. Because this tutorial develops meodels in Pytorch, we have to set the DGL backend to Pytorch.

In [2]:
import dgl
from dgl import DGLGraph

# Load Pytorch as backend
dgl.load_backend('pytorch')

Using backend: pytorch
Using backend: pytorch


Load the rest of necessary libraries.

In [3]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy import sparse as spsp

### Dataset

We use the MovieLens dataset for demonstration because it is commonly used for ecommendation models. in this dataset, there are two types of nodes; users and movies . The movie nodes have three attributes ; year, title and genre. There are ratings between user models and movie does. Each rating has a timestamp. In our recommendation model, we don't consider ratings and timestamps.

__Note:__ It is not necessarily the best dataset to demonstarate the power of GNN for recommendation. We have prepared the dataset simplify the demonstartion.

To run the data preprocessing script, a user nedds to download the English dictionary of the standfordnlp package first,However, the following command only needs to run once.

In [13]:
import stanfordnlp
stanfordnlp.download('en')

Using the default treebank "en_ewt" for language "en".
Would you like to download the models for: en_ewt now? (Y/n)


 Y



Default download directory: /home/tteon/stanfordnlp_resources
Hit enter to continue or type an alternate directory.



Download location: /home/tteon/stanfordnlp_resources/en_ewt_models.zip


100%|██████████| 235M/235M [05:18<00:00, 737kB/s] 



Download complete.  Models saved to: /home/tteon/stanfordnlp_resources/en_ewt_models.zip
Extracting models file for: en_ewt
Cleaning up...Done.


In [4]:
from movielens import MovieLens
data = MovieLens('.')

File not found. Downloading from https://s3.us-east-2.amazonaws.com/dgl.ai/dataset/movielens.tar.gz


  0%|          | 5/3702 [00:00<01:23, 44.19it/s]

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/tteon/stanfordnlp_resources/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/home/tteon/stanfordnlp_resources/en_ewt_models/en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
Done loading processors!
---


100%|██████████| 3702/3702 [00:00<00:00, 80509.13it/s]
/home/tteon/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:942: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [5]:
ratings = data.ratings
user_id = np.array(ratings['user_idx'])
movie_id = np.array(ratings['movie_idx'])
user_movie_spm = spsp.coo_matrix((np.ones((len(user_id),)), (user_id, movie_id)))
num_users, num_movies = user_movie_spm.shape

In [6]:
print('#user-movie iterations:', len(movie_id))
print('#users:', num_users)
print('#movies:', num_movies)

#user-movie iterations: 1000205
#users: 6040
#movies: 3702


We split the dataset into a training set and a testing set

In [7]:
ratings_train = ratings[~(ratings['valid_mask'] | ratings['test_mask'])]
user_latest_item_indices = (ratings_train.groupby('user_id')['timestamp'].transform(pd.Series.max)==ratings_train['timestamp'])
user_latest_item = ratings_train[user_latest_item_indices]
user_latest_item = dict(zip(user_latest_item['user_idx'].values, user_latest_item['movie_idx'].values))

Construct the training, validation and testing dataset

In [8]:
# The traning dataset
user_id = np.array(ratings_train['user_idx'])
movie_id = np.array(ratings_train['movie_idx'])
user_movie_sam = spsp.coo_matrix((np.ones((len(user_id),)), (user_id, movie_id)))
assert num_users == user_movie_spm.shape[0]
assert num_movies == user_movie_spm.shape[1]
train_size = len(user_id)
print('#training size:', train_size)

#training size: 986669


In [10]:
# The validation and testing dataset
users_valid = ratings[ratings['valid_mask']]['user_idx'].values
movies_valid = ratings[ratings['valid_mask']]['movie_idx'].values
users_test = ratings[ratings['test_mask']]['user_idx'].values
movies_test = ratings[ratings['test_mask']]['movie_idx'].values
valid_size = len(users_valid)
test_size = len(users_test)

### The recommendation model

At large, the model first learns item embeddings from the user-item interation dataset and use the item embeddings to recommend users similar items they have purchased. To learn item embeddings, we first need to construct an item similarity graph and train GNN on the item graph.

There are many ways of constructing the item similarity graph. Here we use the __SLIM MODEL__ to learn item similarity and use the learned result to construct the item graph. The resulting graph will have an edge between two items if they are similar and the edge has a weight that represents the simlilarity score.

After the item similarity graph is consturcted, we run a GNN model on it and use the vertex connectivity as the training singnal to train the GNN model. The GNN training procedure is very similar to the link predction task in the precious section.

### Consturct the movie graph with SLIM

SLIM is an item-based recommendation model. when training SLIM on a user-item dataset, it learns an item similarity graph. This similarity graph is the imtem graph we construct for the GNN model.



In [11]:
from SLIM import SLIM, SLIMatrix
from slim_load import read_csr

def gen_slim_graph(user_movie_spm):
    model = SLIM()
    params = {'algo': 'cd', 'nthreads': 2, 'l1r': 1.0 , 'l2r' : 1.0}
    trainmat = SLIMatrix(user_movie_spm.tocsr())
    model.train(params, trainmat)
    model.save_model(modelfname='slim_model.csr', mapfname='slim_map.csr')
    
    #Load the sLIM similarity matrix into DGL. we store the vertex similarity as edge data on DGL.
    movie_spm = read_csr('slim_model.csr')
    print('#edges:', movie_spm.nnz)
    print('most similar:', np.max(movie_spm.data))
    print('most unsimilar:', np.min(movie_spm.data))
    
    # GEnerate DGLGraph
    g = dgl.DGLGraph(movie_spm, readonly=True)
    g.edata['similarity'] = torch.tensor(movie_spm.data, dtype=torch.float32)
    return g

### Construct the co-occurance graph

In [12]:
downsample_factor = 1e-6

def gen_cooccur_graph(user_movie_spm):
    user_id = user_movie_spm.row
    movie_id = user_movie_spm.col
    spm_t = user_movie_spm.transpose()
    movie_deg = spm_t.dot(np.ones((num_users,)))
    movie_ratio = movie_deg / np.sum(movie_deg)
    # 1e-6 is a hyperparameter for this dataset.
    movie_sample_prob = 1 - np.maximum(1 - np.sqrt(downsample_factor / movie_ratio), 0)
    sample_prob = movie_sample_prob[movie_id]
    sample = np.random.uniform(size=(len(movie_id),))
    user_id = user_id[sample_prob > sample]
    movie_id = movie_id[sample_prob > sample]
    print('#samples:', len(user_id))
    user_movie_spm = spsp.coo_matrix((np.ones((len(user_id),)), (user_id, movie_id)))
    print(user_movie_spm.shape)
    movie_deg = spm_t.dot(np.ones((num_users,)))
    print(np.sum(movie_deg == 0))
    
    movie_spm = np.dot(user_movie_spm.transpose(), user_movie_spm)
    #dense_movie = np.sort(movie_spm.todense())
    #topk_movie = dense_movie[:,-50]
    #movie_spm1 = movie_spm >= topk_movie
    
    g = dgl.DGLGraph(movie_spm, readonly=True)
    g.edata['similarity'] = torch.tensor(movie_spm.data, dtype=torch.float32)
    return g

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
topk = 20
def gen_cosine_graph(user_movie_spm):
    movie_spm = cosine_similarity(user_movie_spm.transpose(),dense_output=False)
    
    dense_movie = np.sort(movie_spm.todense())
    topk_movie = dense_movie[:,-topk]
    topk_movie_spm = movie_spm > topk_movie
    topk_movie_spm = movie_spm.multiply(topk_movie_spm)
    g = dgl.DGLGraph(topk_movie_spm, readonly=True)
    g.edata['similarity'] = torch.tensor(topk_moive_spm.data, dtype=torch.float32)
    
    return g

### Generate graphs for training

In [14]:
conv_g = gen_slim_graph(user_movie_spm)
loss_g = gen_cooccur_graph(user_movie_spm)

Learning takes 37.576 secs.
#edges: 307660
most similar: 0.65398
most unsimilar: 0.0
#samples: 49605
(6040, 3702)
0


Construct the item features.

In [15]:
year = np.expand_dims(data.movie_data['year'], axis=1)
genre = data.movie_data['genre']
title = data.movie_data['title']
features = torch.tensor(np.concatenate((genre, title), axis=1), dtype=torch.float32)
print('#featuers:', features.shape[1])
in_feats = features.shape[1]

#featuers: 4124


### GNN models

We run GNN on the item graph to compute item embeddings. In this tutorial. we use a sustomized __GraphSage__ model to compute node embeddings. The original GraphSaeg performs the following computations on every node v in the graph:

The original GraphSage model treats each neighbor equally. However, the SLIM model learns the item similarity based on the user-item iteration. The GNN model should take the similarity into account. Thus, we customize the GraphSage model in the following fashion. Instead of aggregating all neighbors equally, we aggregate neighbors embeddings rescaled by the similarity on the edges. Thus, the aggregation step is defined as follows:

The GNN model has multiple layers. Ine ach layers, a vertex accesses its direct neighbors. When we stack k layers in a model, a node v access neighbors within k hops. The output of the GNN model is node embeddings that represent the nodes and all information in the k-hop neighborhood.

We implement the computation in each layer of the customized GraphSage model in SAGEConv and implement the multi-layer model in GraphSAGEModel.

In [18]:
from sageconv import SAGEConv

class GraphSAGEModel(nn.Module):
    def __init__(self,
                 in_feats,
                 n_hidden,
                 out_dim,
                 n_layers,
                 activation,
                 dropout,
                 aggregator_type):
        super(GraphSAGEModel, self).__init__()
        self.layers = nn.ModuleList()

        # input layer
        self.layers.append(SAGEConv(in_feats, n_hidden, aggregator_type,
                                    feat_drop=dropout, activation=activation))
        # hidden layers
        for i in range(n_layers - 1):
            self.layers.append(SAGEConv(n_hidden, n_hidden, aggregator_type,
                                        feat_drop=dropout, activation=activation))
        # output layer
        self.layers.append(SAGEConv(n_hidden, out_dim, aggregator_type,
                                    feat_drop=dropout, activation=None))

    def forward(self, g, features):
        h = features
        for layer in self.layers:
            h = layer(g, h, g.edata['similarity'])
        return h

### Train Item Embeddings

we train the item embeddings with the edges in the item graph as the training signal. This step is very similar to the link prediction task in the __basic applications.__

Becaues the MovieLens dataset has sparse features (both genre and title are stored as multi-hot encoding). The sparse features have many dimensions. To run GNN on the item features, we first create an encoding layer to project the sparse features to a lower dimension.

In [28]:
class EncodeLayer(nn.Module):
    def __init__(self, in_feats, num_hidden):
        super(EncodeLayer, self).__init__()
        self.proj = nn.Linear(in_feats, num_hidden)
        
    def forward(self, feats):
        return self.proj(feats)

We first use GraphSage as the base model to compute node embeddings.

In [20]:
# Model hyperparameters
n_hidden = 64
n_layers = 1
dropout = 0.5
aggregator_type = 'sum'

# create GraphSAGE model
gconv_model = GraphSAGEModel(n_hidden,
                            n_hidden,
                            n_hidden,
                            n_layers,
                            F.relu,
                            dropout,
                            aggregator_type)

We simply use node connectivity as the training signal :nodes connected by edges are similar, while nodes not connected by edges are dissimilar.

To train such a model, we need to deploy negative sampling to construct negative samples. A positive sample is an edge that exist in the original graph, while a negative sample is a pair of nodes that don't have an edge between them in the graph. We usually train on each positive sample with multiple negative samples.

After having the node embeddings, we compute the similarity scores on positive samples and negative samples. WE construct the following loss function on a positive sample and the corresponding negative samples:

With this loss, training should increase the simliarity scores on the positive samples and decrease the similarity scores on negative samples.

In [26]:
# NCE loss
def NCE_loss(pos_score, neg_score, neg_sample_size):
    pos_score = F.logsigmoid(pos_score)
    neg_score = F.logsigmoid(-neg_score).reshape(-1, neg_sample_size)
    return -pos_score - torch.sum(neg_score, dim=1)

class GNNRec(nn.Module):
    def __init__(self, gconv_model):
        super(GNNRec, self).__init__()
        self.encode = EncodeLayer(in_feats, n_hidden)
        self.gconv_model = gconv_model
        
    def forward(self, conv_g, loss_g, features, neg_sample_size):
        emb = self.encode(features)
        emb = self.gconv_model(conv_g, emb)
        pos_g, neg_g = edge_sampler(loss_g, neg_sample_size, return_false_neg=False)
        pos_score = score_func(pos_g, emb)
        neg_score = score_func(neg_g, emb)
        return torch.mean(NCE_loss(pos_score, neg_score, neg_sample_size) * pos_g.edata['weight'])
    

DGL provides an edge sampler EdgeSampler, which selects positive edge samples and negative edge samples efficiently. Thus, we can use it to generate positive samples and negative samples for link prediction. EdgeSampler generates neg_sample_size negative edges by corrupting the head or the tail node of a positive edge with some randomly sampled nodes.

edge_sampler samples one tenth of the edges in the graph as positive edges. It reutrns a positive subgraph and a negative subgraph. The positivie subgraph contains all positive edges sampled from the graph g, while the negative subgraph contains all negative edges contructed by the edge sampler.

In [30]:
def edge_sampler(g, neg_sample_size, edges=None, return_false_neg=True):
    sampler = dgl.contrib.sampling.EdgeSampler(g, batch_size=int(g.number_of_edges()/10), seed_edges = edges, neg_sample_size = neg_sample_size,
                                              negative_mode='tail',
                                               shuffle=True,
                                              return_false_neg=return_false_neg)
    sampler = iter(sampler)
    pos_subg, neg_subg = next(sampler)
    pos_subg.edata['weight'] = g.edata['similarity'][pos_subg.parent_eid]
    return pos_subg, neg_subg

After having the positive edge subgraph and the negative edge subgraph, we can now compute the similarity on the positive edge samples and negative edge samples.

In this tutorial, we use dot-product similarity to measure the similarity between two nodes.

In [23]:
def score_func(g, emb):
    src_nid, dst_nid = g.all_edges(order='eid')
    # Get the node Ids in the parent graph.
    src_nid = g.parent_nid[src_nid]
    dst_nid = g.parent_nid[dst_nid]
    # Read the node embeddings of the source nodes and destination nodes.
    pos_heads = emb[src_nid]
    pos_tails = emb[dst_nid]
    # cosine similarity
    return torch.sum(pos_heads * pos_tails, dim=1)

We evaluate the performance of the trained item embeddings in the item-based recommendation task. We use the last item that a user purchased to represent the user and compute the similarity between the last item and a list of item( a item the user will purchase and a set of randomly sampled items). We calculate the ranking of the item that will be purchased among the list of items.

In [24]:
def RecEvaluate(model, g, features, users_eval, movies_eval, neg_sample_size):
    gconv_model.eval()
    with torch.no_grad():
        emb = model.encode(features)
        emb = model.gconv_model(g, emb)
        hits_10s = []
        # evaluate one user-item interaction at a time
        for u, i in zip(users_eval, movies_eval):
            I_q = user_latest_item[u]
            I = torch.cat([torch.LongTensor([i]), torch.LongTensor(data.neg_valid[u])])
            Z_q = emb[I_q]
            Z = emb[I]
            score = (Z_q[None, :] * Z).sum(1).cpu().numpy()
            rank = stats.rankdata(-score, 'min')
            hits_10s.append(rank[0] <= 10)
        print('HITS@10:{:.4f}'.format(np.mean(hits_10s)))
        return np.mean(hits_10s)

# Training loop.

In [33]:
# Model for link prediction
model = GNNRec(gconv_model)

# Training hyperparameters
weight_decay = 5e-4
n_epochs = 20
lr = 1e-3
neg_sample_size = 40

# use optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# initialize graph
dur = []
prev_acc = 0
for epoch in range(n_epochs):
    model.train()
    loss = model(conv_g, loss_g, features, neg_sample_size)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print("Epoch {:05d} | Loss {:.4f}".format(epoch, loss.item()))
    
    acc = RecEvaluate(model, conv_g, features, users_valid, movies_valid, neg_sample_size)
    # We have an early stop
    if epoch > 8 and acc <= prev_acc:
        break
    prev_acc = acc
    
print()
# Let's save the trained node embeddings.
RecEvaluate(model, conv_g, features, users_test, movies_test, neg_sample_size)

Epoch 00000 | Loss 94.8204
HITS@10:0.2932
Epoch 00001 | Loss 65.5102
HITS@10:0.2718
Epoch 00002 | Loss 50.8380
HITS@10:0.2934
Epoch 00003 | Loss 45.4391
HITS@10:0.3114
Epoch 00004 | Loss 42.8007
HITS@10:0.3142
Epoch 00005 | Loss 40.1818
HITS@10:0.3141
Epoch 00006 | Loss 38.4314
HITS@10:0.3230
Epoch 00007 | Loss 37.4669
HITS@10:0.3405
Epoch 00008 | Loss 36.6479
HITS@10:0.3460
Epoch 00009 | Loss 36.3470
HITS@10:0.3420

HITS@10:0.4075


0.4074832464631422